In [2]:
#Manejo de Datos
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import boxcox


#Machine learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import Callback

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix

#Librerias estandar (Extras)
import re
import os
import time
import random
from datetime import datetime
import glob

In [3]:
"""
DIRECTORIOS DEL PROYECTO
"""
path_base = 'C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/GPUTesis'   # Directorio del proyecto
path_imagenes = 'F:/GOES/'      

products = ['C07','C08','C13']
times   = ['10','20','30','40','50','00']

In [4]:
def crearModelo2D(p,run):    
    # Imagen
    input_1 = tf.keras.layers.Input(shape=(p['margen'][run],p['margen'][run],p['canales'][run]))
    
    # Convulutional layers
    rescaling = tf.keras.layers.Rescaling(1./65536)(input_1)
    conv2d_1 = tf.keras.layers.Conv2D(128, kernel_size=3,padding='same',activation=tf.keras.activations.relu)(rescaling)
    mxPool_1 = tf.keras.layers.MaxPooling2D()(conv2d_1)
    dropout_1  = tf.keras.layers.Dropout(0.2)(mxPool_1)
    
    conv2d_2 = tf.keras.layers.Conv2D(64, kernel_size=3,padding='same',activation=tf.keras.activations.relu)(dropout_1)
    mxPool_2 = tf.keras.layers.MaxPooling2D()(conv2d_2)
    dropout_2  = tf.keras.layers.Dropout(0.1)(mxPool_2)
    
    #conv2d_3 = tf.keras.layers.Conv2D(32, kernel_size=3,padding='same',activation=tf.keras.activations.relu)(dropout_1)
    #mxPool_3 = tf.keras.layers.MaxPooling2D()(conv2d_3)
    #dropout_3  = tf.keras.layers.Dropout(0.2)(mxPool_3)
    
    #conv2d_4 = tf.keras.layers.Conv2D(64, kernel_size=3,padding='same',activation=tf.keras.activations.relu)(dropout_3)
    #mxPool_4 = tf.keras.layers.MaxPooling2D()(conv2d_4)
    #dropout_4  = tf.keras.layers.Dropout(0.2)(mxPool_4)
    
    conv2d_5 = tf.keras.layers.Conv2D(32, kernel_size=3,padding='same',activation=tf.keras.activations.relu)(dropout_2)
    
    
    # Flatten layer :
    flatten = tf.keras.layers.Flatten()(conv2d_5)
    
    final = flatten
    listConcat = [flatten]
    listInputs = [input_1]
    
    if len(p['inputs'])>1:
        #Agregamos los otros atrbutos        
        for attr in p['inputs'][1:]:
            # The other input
            input_x = tf.keras.layers.Input(shape=(1,))
            listConcat.append(input_x)
            listInputs.append(input_x)

            
        # Concatenate
        final = tf.keras.layers.Concatenate()(listConcat)
        
    dense_1 = tf.keras.layers.Dense(units=32, activation=tf.keras.activations.relu)(final)
    #dense_2 = tf.keras.layers.Dense(units=16, activation=tf.keras.activations.relu)(dense_1)
    dense_3 = tf.keras.layers.Dense(units=32, activation=tf.keras.activations.relu)(dense_1)
    
        
    # output
    if p['redTipo'] == 'regresion':
        output = tf.keras.layers.Dense(units=1)(dense_3)
        dimOutput = 1
    elif p['redTipo'] == 'clasificacion':
        output = tf.keras.layers.Dense(units=1,activation=tf.keras.activations.sigmoid)(dense_3)#units=1, activation=tf.keras.activations.relu)(dense_3)
        dimOutput = 2
    else:
        print(f"No se pudo crear el modelo outputs no esta bien definido {p['redTipo']}")
        return -1      
    

    full_model = tf.keras.Model(inputs=listInputs, outputs=[output])
        
    #print(full_model.summary())
    return full_model

In [5]:
def crearModelo3D(p,run):        
    # Imagen
    input_1 = tf.keras.layers.Input(shape=(p['tiempos'][run],p['margen'][run],p['margen'][run],p['canales'][run]))
    
    # Convulutional layers
    rescaling = tf.keras.layers.Rescaling(1./65536)(input_1)
    conv3d_1 = tf.keras.layers.Conv3D(128, kernel_size=3,padding='same',activation=tf.keras.activations.relu)(rescaling)
    #mxPool_1 = tf.keras.layers.MaxPooling3D()(conv3d_1)
    dropout_1  = tf.keras.layers.Dropout(0.2)(conv3d_1)
    
    conv3d_2 = tf.keras.layers.Conv3D(64, kernel_size=3,padding='same',activation=tf.keras.activations.relu)(dropout_1)
    mxPool_2 = tf.keras.layers.MaxPooling3D()(conv3d_2)
    dropout_2  = tf.keras.layers.Dropout(0.1)(mxPool_2)
    
    #conv2d_3 = tf.keras.layers.Conv3D(32, kernel_size=3,padding='same',activation=tf.keras.activations.relu)(dropout_1)
    #mxPool_3 = tf.keras.layers.MaxPooling3D()(conv2d_3)
    #dropout_3  = tf.keras.layers.Dropout(0.2)(mxPool_3)
    
    #conv2d_4 = tf.keras.layers.Conv3D(64, kernel_size=3,padding='same',activation=tf.keras.activations.relu)(dropout_3)
    #mxPool_4 = tf.keras.layers.MaxPooling3D()(conv2d_4)
    #dropout_4  = tf.keras.layers.Dropout(0.2)(mxPool_4)
    
    conv3d_5 = tf.keras.layers.Conv3D(32, kernel_size=3,padding='same',activation=tf.keras.activations.relu)(dropout_2)
    
    
    # Flatten layer :
    flatten = tf.keras.layers.Flatten()(conv3d_5)
    
    final = flatten
    listConcat = [flatten]
    listInputs = [input_1]
    
    if len(p['inputs'])>1:
        #Agregamos los otros atrbutos        
        for attr in p['inputs'][1:]:
            # The other input
            input_x = tf.keras.layers.Input(shape=(1,))
            listConcat.append(input_x)
            listInputs.append(input_x)

            
        # Concatenate
        final = tf.keras.layers.Concatenate()(listConcat)
        
    dense_1 = tf.keras.layers.Dense(units=32, activation=tf.keras.activations.relu)(final)
    #dense_2 = tf.keras.layers.Dense(units=16, activation=tf.keras.activations.relu)(dense_1)
    dense_3 = tf.keras.layers.Dense(units=32, activation=tf.keras.activations.relu)(dense_1)
    
        
    # output
    if p['redTipo'] == 'regresion':
        output = tf.keras.layers.Dense(units=1)(dense_3)
        dimOutput = 1
    elif p['redTipo'] == 'clasificacion':
        output = tf.keras.layers.Dense(units=1,activation=tf.keras.activations.sigmoid)(dense_3)#units=1, activation=tf.keras.activations.relu)(dense_3)
        dimOutput = 2
    else:
        print(f"No se pudo crear el modelo outputs no esta bien definido {p['redTipo']}")
        return -1      
    

    full_model = tf.keras.Model(inputs=listInputs, outputs=[output])
        
    #print(full_model.summary())
    return full_model

In [6]:
def crearModelo(params,run):    
    if params['meanMatrizImagen']:
        print(f"Creando modelo 2D")
        print(f"HP :(T:{params['tiempos'][run]} - M:{params['margen'][run]} - C:{params['canales'][run]}) y  tipo ({params['redTipo']})")
        modelo = crearModelo2D(params,run)
    else:
        print(f"Creando modelo 3D")
        print(f"HP :(T:{params['tiempos'][run]} - M:{params['margen'][run]} - C:{params['canales'][run]}) y  tipo ({params['redTipo']})")
        modelo = crearModelo3D(params,run)

    return modelo

In [7]:
#Transformamos un filename tensor en una imagen
def read_png_file(item, value, p,run, path_base, products, times):
    # imagenData[0] = XO     # imagenData[1] = XA     # imagenData[2] = Fecha
    imagenData = tf.strings.split(item['imagen'], sep='--')
    size = int(p['margen'][run] / 2)

    timeJoin = []
    for j in range(p['tiempos'][run]-1,-1,-1):
        filename = path_base + 'PNG/' + imagenData[2] + '/' + imagenData[2] + '_' + str(j) + '.png'
        
        image_string = tf.io.read_file(filename)

        img_decoded = tf.io.decode_png(image_string, dtype=tf.uint16, channels=3)
        
        
                
        timeJoin.insert(0,img_decoded[int(imagenData[1]) - size:int(imagenData[1]) + size,
                                      int(imagenData[0]) - size:int(imagenData[0]) + size,
                                      0:p['canales'][run]])
 
        
    if p['tiempos'][run]==1:
        imagenData = tf.reshape(timeJoin[0],(p['margen'][run],p['margen'][run],p['canales'][run]))
    else:
        if p['meanMatrizImagen']:        
            img = tf.reduce_mean( timeJoin , axis=0 )
            imagenData = tf.reshape(img,(p['margen'][run],p['margen'][run],p['canales'][run]))
        else:
            img = tf.stack(timeJoin, axis=0)
            imagenData = tf.reshape(img,(p['tiempos'][run],p['margen'][run],p['margen'][run],p['canales'][run]))
        
    
    
    if len(p['inputs']) == 1:
        return imagenData, int(value)
    
    item['imagen'] = imagenData
    itemL = []
    for inpL in p['inputs']:
        itemL.append(item[inpL])
    
    return tuple(itemL), int(value)

In [8]:
def splitDataset(p, run, ds, path_imagenes, products, times,val_split= 0.2):
    #Dataset de etnrenamiento    
    dataset  = ds        
        
    # Escojemos una fraccion del dataset si se ha indicado   
    if p['dataset']:
        train, test = train_test_split(dataset.sample(frac=p['dataset']), test_size=val_split, shuffle=True)
    else:
        train, test = train_test_split(dataset, test_size=val_split, shuffle=True)        
        
    print(f'Tamaño del dataset: Train {len(train)}  - Val {len(test)}')
    
    inputsList = {}
    for inp in p['inputs']:
        inputsList[inp] = train[inp].tolist()              
        
    train_dataset = tf.data.Dataset.from_tensor_slices(((inputsList),train[p['outputs']].tolist()))           
    val_dataset = tf.data.Dataset.from_tensor_slices(((inputsList),train[p['outputs']].tolist()))     
    
    train_dataset = train_dataset.map(lambda x ,y : read_png_file(x,y,p,run,path_imagenes,products,times))
    val_dataset = val_dataset.map(lambda x ,y : read_png_file(x,y,p,run,path_imagenes,products,times))
    
    train_dataset = train_dataset.batch(p['batch'])#.cache().prefetch(tf.data.AUTOTUNE)
    val_dataset = val_dataset.batch(p['batch'])#.prefetch(tf.data.AUTOTUNE)  
    
    return train_dataset, val_dataset

In [9]:
def evaluarModelo(dsPruebas,modelo, p_train):
    inputsList = {}
    for inp in p_train['inputs']:
        inputsList[inp] = dsPruebas[inp].tolist()       

    dsP = tf.data.Dataset.from_tensor_slices(((inputsList),dsPruebas[p_train['outputs']].tolist()))      
    dsP = dsP.map(lambda x ,y : read_png_file(x,y,p_train,0,path_imagenes,products,times))
    dsP = dsP.batch(p_train['batch'])#.prefetch(tf.data.AUTOTUNE)  
    
    hist = modelo.evaluate(dsP)
    return hist

In [17]:
def getModelbyID(dsPruebas, path_base, idModel, _2D = False, inputs=['imagen', 'dato']):
    # Buscamos el modelo en el path_base
    os.chdir(f'{path_base}/Archivos/Modelos')
    listFiles = list(glob.glob('**/**/**/*.hdf5'))
    print(f'Cantidad de modelos a buscar: {len(listFiles)}')  
        
    
    models = [x for x in listFiles if str(idModel) in x]
    print(models)
    print(idModel)
    if len(models) > 0:
        redTipo = models[0].split('\\')[0]

        # Buscamos sus parametros
        pathStats = f'{path_base}/Archivos/Reportes/Entrenamiento/{redTipo}/Reporte-{redTipo}.csv'
        try:
            stats = pd.read_csv(pathStats)        
            modelStats = stats[stats['idModel']==idModel]
            print(f'MODELO ENCONTRAOD : {modelStats}')
        except:
            print('No se pudo encontrar el archivo de stats o leer el id del modelo en el archivo stats')
            print(f'Archivo leido : {pathStats}')
            return -1


        C,T,M = modelStats['HP'].iloc[0].split('_')        
        p_train = {         
                # Datos del modelo
              'redTipo'  : redTipo.lower(), 
              'inputs'   : inputs, 
              'meanMatrizImagen' : _2D, 
              'outputs'  : 'clase', 
              'num_class': 2,

               # Hiper parametros 
              'canales'  : [int(C)],
              'tiempos'  : [int(T)],
              'margen'   : [int(M)],
              'runs'     : 1,
            
              # Entrenamiento
              'batch'    : 32,
              'lr'       : 0.001,
         }
        
        modelo = crearModelo(p_train,0)
        modelo.load_weights(models[0])
        metricas = getMetrics(redTipo.lower(),p_train['lr'], 7)
        modelo.compile(optimizer=metricas['optimizer'],loss=metricas['loss_fn'],metrics=metricas['metrics'],)
        
        hist = evaluarModelo(dsPruebas,modelo, p_train)
        return modelo, p_train, hist
        
    else:
        print(f'No se encontro el modelo con id {idModel} en la PC')
        return -1
    
    return -1

In [18]:
def getMetrics(modelType, lr, paciencia):
    
    if modelType == 'clasificacion':    
        optimizer = keras.optimizers.Adam(learning_rate=lr) 
        
        #BinaryCrossentropy() #CategoricalCrossentropy()      
        loss_fn= keras.losses.BinaryCrossentropy()
        train_acc_metric = keras.metrics.BinaryCrossentropy()
        val_acc_metric = keras.metrics.BinaryCrossentropy()
        if paciencia:
            early_stopping = keras.callbacks.EarlyStopping(monitor="val_acc", patience=paciencia, mode="max")  
 
        
        metrics = ['acc', keras.metrics.TruePositives(),
                         keras.metrics.TrueNegatives(),
                         keras.metrics.FalsePositives(),
                         keras.metrics.FalseNegatives()]
        

    elif modelType == 'regresion':
        optimizer = keras.optimizers.Adam(learning_rate=lr)
        loss_fn=keras.losses.MeanSquaredError()
        train_acc_metric = keras.metrics.MeanSquaredError()
        val_acc_metric = keras.metrics.MeanSquaredError()
        if paciencia:
            early_stopping = keras.callbacks.EarlyStopping(monitor="val_mse", patience=paciencia, mode="max")                                            
        metrics = ['mse']
        
        
    else:
        print('No se pudo crear las metricas')
        return -1    
         
        
    logs = Callback()
    callbacks = [logs]                     
    if paciencia:
        callbacks.append(early_stopping)
        
    metrics = {'optimizer': optimizer, 'loss_fn':loss_fn,'train_acc_metric': train_acc_metric,
               'val_acc_metric': val_acc_metric, 'metrics': metrics,'callbacks': callbacks}
    
    return metrics
        

In [19]:
# leemos el DS
print('Pruebas para modelos convulucional 3D')
pruebasFile = f'{path_base}/Archivos/Dataset/Pruebas/CLASEV2-C3_PruebasDS.csv'
#pruebasFile = f'{path_base}/Archivos/Dataset/SplittedV2/CLASE_TrainDS_2.csv'
#pruebasFile = f'{path_base}/Archivos/Dataset/CLASE_TrainDS.csv'
dsPruebas = pd.read_csv(pruebasFile)
print(f'Cantidad de pruebas : {len(dsPruebas)}')  
dsPruebas.head(2)

Pruebas para modelos convulucional 3D
Cantidad de pruebas : 367


,Unnamed: 0,nombre,codigo,XO,XA,longitud,latitud,altura,dato,90%,99%,75%,fecha,flag,flagV2,imagen,clase
0,0,LA FORTUNA,107131,286,502,-78.40242,-7.67042,3343.0,0.0,0.4,5.357,0.0,2021-12-27-11,C0000002,C01,286--502--2021-12-27-11,1
1,1,TUNEL CHOTANO,47E1F02E,265,440,-78.78432,-6.54049,2027.0,0.0,0.1,3.939,0.0,2021-12-11-23,M0000002,C01,265--440--2021-12-11-23,0


In [20]:
dsPruebasS = dsPruebas[dsPruebas['flagV2']=='D02']
dsPruebasS = dsPruebasS[dsPruebas['flag']=='C0000002']
len(dsPruebasS)
dsPruebasS['flag'].unique()

C:\Users\Shounen\AppData\Local\Temp\ipykernel_108\751691209.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dsPruebasS = dsPruebasS[dsPruebas['flag']=='C0000002']


array(['C0000002'], dtype=object)

In [21]:
#dsPruebas = dsPruebas[dsPruebas['flag']=='C0000002']
dsPruebas = dsPruebas[dsPruebas['dato']!=0]

In [24]:
modelo, p_train, hist = getModelbyID(dsPruebas, path_base, '20220429_180712' , _2D = True)
#20220427_151059 
#20220429_180712

Cantidad de modelos a buscar: 125
['Clasificacion\\TesisDiego3D_V3\\CLASE_TrainDS_15\\Model_clasificacion_clase_20220429_180712.hdf5']
20220429_180712
MODELO ENCONTRAOD : Empty DataFrame
Columns: [Unnamed: 0, dsDir, idModel, HP, DS_len, dsName, epoca, val_TN, val_TP, val_FP, val_FN, val_acc, TNR, TPR, PPV, NPV, F1]
Index: []


IndexError: single positional indexer is out-of-bounds

In [ ]:
hist

In [ ]:
dsPruebas['dato'].hist()
 
print(f'Cantidad total: {dsPruebas["dato"].count()}')
print(f'Cantidad ceros: {dsPruebas[dsPruebas["dato"]==0]["dato"].count()}')

In [28]:
filename = 'F:/GOES/PNG/2020-01-01-04/2020-01-01-04_1.png'
image_string = tf.io.read_file(filename)
img_decoded = tf.io.decode_png(image_string, dtype=tf.uint16, channels=3)       
img_decoded = tf.image.flip_left_right(img_decoded)

#plt.imshow(img_decoded)
